In [2]:
import pandas as pd
from darts import TimeSeries

ModuleNotFoundError: No module named 'darts'

In [ ]:
from darts.models import AutoARIMA

In [ ]:
import pandas as pd
from pandas import Series
from pandas.tseries.offsets import DateOffset
import numpy as np
import scipy
from scipy import stats
import scipy.stats as scs
import itertools
from scipy.special import boxcox, inv_boxcox

In [ ]:
import statsmodels
import statsmodels.api as sm
import statsmodels.stats as sms
from statsmodels.stats.stattools import jarque_bera
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.tsa.api as smt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics import tsaplots
import pmdarima as pm

In [ ]:
from datetime import datetime, timedelta, date

In [ ]:
import prophet
from prophet import Prophet

In [ ]:
import sys
import warnings
import matplotlib.cbook
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=matplotlib.cbook.mplDeprecation)


import pandas as pd
from pandas import Series
from pandas.tseries.offsets import DateOffset
import numpy as np
import scipy
from scipy import stats
import scipy.stats as scs
import itertools
from scipy.special import boxcox, inv_boxcox


import matplotlib.pyplot as plt
import matplotlib as m
import seaborn as sns

# Data Source
import yfinance as yf

# Stats models
import statsmodels
import statsmodels.api as sm
import statsmodels.stats as sms
from statsmodels.stats.stattools import jarque_bera
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.tsa.api as smt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics import tsaplots
import pmdarima as pm

# Date tools
from datetime import datetime, timedelta, date

# Prophet model
import prophet
from prophet import Prophet

# Imports for model metrics and performance
import math
from math import sqrt 
import sklearn
from sklearn.metrics import mean_squared_error
import plotly as py
import plotly.express as px
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
import re

In [ ]:
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf

from sklearn.metrics import mean_squared_error

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams["figure.figsize"] = (14,6)
plt.style.use('fivethirtyeight')
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
m.rcParams['axes.labelsize'] = 14
m.rcParams['xtick.labelsize'] = 12
m.rcParams['ytick.labelsize'] = 12
m.rcParams['text.color'] = 'k'

In [ ]:
data_path = ("../raw_data/data_daily.csv")
data = pd.read_csv(data_path)
data["timestamp"] = pd.to_datetime(data['timestamp'])

data.drop_duplicates(subset='timestamp', inplace=True)

data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum().sum()

In [ ]:
#data=data.dropna()

In [ ]:
data.duplicated().sum()

In [ ]:
#data = data.drop_duplicates()

In [ ]:
#had to remove duplicates
#series = TimeSeries.from_dataframe(data,"timestamp", "close", freq='H')

# TRAIN SPLIT

In [ ]:
train_size = 0.7
index = round(train_size*data.shape[0])

data_train = data.iloc[:index]
data_test = data.iloc[index:]

In [ ]:
data_train.shape

In [ ]:
data_test.shape

In [ ]:
y_train=data_train[['timestamp', 'close']]
X_train=data_train.drop(columns=['timestamp', 'close'])
X_train

In [ ]:
y_train

In [ ]:
y_test=data_test[['timestamp', 'close']]
X_test=data_test.drop(columns=['timestamp', 'close'])
X_test

In [ ]:
y_test

# ETS(baseline)

In [ ]:
from darts.models import ExponentialSmoothing

In [ ]:
data

In [ ]:
data.info()

In [ ]:
#data.reset_index(inplace=True)

In [ ]:
data

In [ ]:
data['log_close'] = np.log(data['close'])

In [ ]:
series = TimeSeries.from_dataframe(data, 'timestamp', "log_close",freq='1D')

In [ ]:
train, val = series.split_before(pd.Timestamp('2023-01-01 00:00:00'))

In [ ]:
data['timestamp'].duplicated().sum()

In [ ]:
model = ExponentialSmoothing()
model.fit(train)
prediction = model.predict(1)#, num_samples=1000)

In [ ]:
series[:1300].plot()
prediction.plot()

In [ ]:
len(train)

In [ ]:
prediction = model.predict(len(val))

In [ ]:
import matplotlib.pyplot as plt

series.plot()
prediction.plot(label="forecast", low_quantile=0.05, high_quantile=0.95)
plt.legend()

In [ ]:
from darts.ad import KMeansScorer

scorer = KMeansScorer(k=2, window=5)
scorer.fit(train)
anom_score = scorer.score(val)

In [ ]:
from darts.ad import QuantileDetector

detector = QuantileDetector(high_quantile=0.99)
detector.fit(scorer.score(train))
binary_anom = detector.detect(anom_score)

In [ ]:
import matplotlib.pyplot as plt

series.plot()
(anom_score / 2. - 100).plot(label="computed anomaly score", c="orangered", lw=3)
(binary_anom * 45 - 150).plot(label="detected binary anomaly", lw=4)

In [ ]:
from darts.metrics import mape

In [ ]:
score=mape(val,prediction)

In [ ]:
score

In [ ]:
from darts.metrics import mase

In [ ]:
score1=mase(val,prediction,train)

In [ ]:
score1

In [ ]:
model = ExponentialSmoothing()

backtest = model.historical_forecasts(series,start=0.8, forecast_horizon=1)

In [ ]:
series[1000:].plot()
backtest.plot(label="forecast", low_quantile=0.05, high_quantile=0.95)
plt.legend()

Choosing the model for forecasting

Gridsearch best model.best params

# ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
data.set_index('timestamp',inplace=True)

In [ ]:
data

In [ ]:
type(data.index)

Multiplicative Decomposition 

In [ ]:
indexed_data=data[:1000]

In [ ]:
indexed_data

In [ ]:
result_mul = seasonal_decompose(data['close'], model='multiplicative',period=120)

In [ ]:
pwd!

In [ ]:
#Removing seasons
data['deseasonalized'] = data.close.values/result_mul.seasonal

In [ ]:
#plt.figure(figsize=(15,4)); plt.subplot(1,2,1); plt.plot(data.deseasonalized);
#plt.title('close price lin', fontsize=16);

In [ ]:
#Removing trend
data['linearized'] = np.log(data['deseasonalized'])

In [ ]:
#plt.subplot(1,2,2); plt.plot(data['linearized'])
#plt.title('close price lin', fontsize=16);

In [ ]:
# 1st Differencing
y_diff = data['linearized'].diff().dropna()

# 2nd Differencing
y_diff_diff = data['linearized'].diff().diff().dropna()

In [ ]:
print('p-value zero-diff: ', adfuller(data['linearized'])[1])
print('p-value first-diff: ', adfuller(data['linearized'].diff().dropna())[1])
print('p-value second-diff: ', adfuller(data['linearized'].diff().diff().dropna())[1])

In [ ]:
from pmdarima.arima.utils import ndiffs
ndiffs(data['linearized'])

In [ ]:
arima = ARIMA(data['linearized'], order=(2, 1, 1), trend='t')
arima = arima.fit()

In [ ]:
arima.summary()

parameters

In [ ]:
smodel = pm.auto_arima(df['linearized'],
                       start_p=1, max_p=2,
                       start_q=1, max_q=2,
                       trend='t',
                       seasonal=False,
                       trace=True)

In [ ]:
aarima_model = AutoARIMA(
    start_p=1, start_q=1,
    max_p=5, max_q=5,
    m=1,
    d=1,
    seasonal=False,
    start_P=0,
    D=None,
    trace=True,
    error_action='ignore',
    suppress_warnings=True, stepwise=True
)

aarima_model.fit(train)

In [ ]:
aarima_model.model_params

In [ ]:
data['linearized']

In [ ]:
prediction = aarima_model.predict(len(val))

In [ ]:
import matplotlib.pyplot as plt

series.plot(label='actual')
prediction.plot(label='forecast', lw=3)
plt.legend()

In [ ]:
train = data['linearized'][0:25000]
val = data['linearized'][25000:]

# Building model
arima = ARIMA(train, order=(2, 1, 1), trend='t')  
arima = arima.fit()


forecast = arima.forecast(len(val), alpha=0.05)  # 95% confidence


forecast_results = arima.get_forecast(len(val), alpha=0.05)
forecast = forecast_results.predicted_mean
confidence_int = forecast_results.conf_int().values

In [ ]:
#recomposition back to timeseries
forecast_recons = np.exp(forecast) * result_mul.seasonal[25000:]
train_recons = np.exp(train) * result_mul.seasonal[0:25000]
val_recons = np.exp(val) * result_mul.seasonal[25000:]
lower_recons = np.exp(confidence_int)[:, 0] * result_mul.seasonal[25000:]
upper_recons = np.exp(confidence_int)[:, 1] * result_mul.seasonal[25000:]

In [ ]:
residuals = pd.DataFrame(arima.resid)

fig, ax = plt.subplots(1,2, figsize=(16,3))
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1]);

In [ ]:
best_order = smodel.order

In [ ]:
print(np.exp(ARIMA(log_close,order=best_order).fit().get_forecast(1).predicted_mean))

# SARIMA

Train split

In [ ]:
data['log'] = np.log(data.close)

In [ ]:
train = data.log[0:25000]
val = data.log[25000:]

In [ ]:
smodel = pm.auto_arima(train, seasonal=True, m=12, 
                       start_p=0, max_p=1, max_d=1, start_q=0, max_q=1,
                       start_P=0, max_P=2, max_D=1, start_Q=0, max_Q=2, 
                       trace=True, error_action='ignore', suppress_warnings=True)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX


sarima = SARIMAX(train, order=(1, 1, 1), seasonal_order=(2, 0, 0, 12))
sarima = sarima.fit(maxiter=75)

results = sarima.get_forecast(len(val), alpha=0.05)
forecast = results.predicted_mean
confidence_int = results.conf_int()

In [ ]:
forecast_recons = pd.Series(np.exp(forecast), index=val.index)
lower_recons = np.exp(confidence_int['lower log']).values
upper_recons = np.exp(confidence_int['upper log']).values

In [ ]:
sarima.summary()

# PROPHET

In [ ]:
df = data.reset_index().rename(columns={'timestamp': 'ds', 'close':'y'})
df.head()

In [ ]:
from prophet import Prophet

train = df.iloc[:28000]
val = df.iloc[28000:]

model = Prophet(seasonality_mode='multiplicative')

model.fit(train)

In [ ]:
forecast_val = model.predict(val)
forecast_val.head()

In [ ]:
model.plot(forecast_val);

In [ ]:
# Predict the next 50 datapoints
future = model.make_future_dataframe(periods=50, freq='1h')
forecast_future = model.predict(future)
forecast_future.head()

In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric

initial = f'{5 * 365} days' 
period = f'{5 * 365} days'
horizon = '365 days'
df_cv = cross_validation(model, initial=initial, period=period, horizon=horizon)